In [1]:
import numpy as np

# package for 3d visualization
from itkwidgets import view                              
from aicssegmentation.core.visual import seg_fluo_side_by_side,  single_fluorescent_view, segmentation_quick_view
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16, 12]

# package for io 
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter

# function for core algorithm
from aicssegmentation.core.vessel import filament_3d_wrapper
from aicssegmentation.core.pre_processing_utils import intensity_normalization, edge_preserving_smoothing_3d, image_smoothing_gaussian_3d
from skimage.morphology import remove_small_objects  
from aicssegmentation.core.vessel import vesselness3D
from aicssegmentation.core.seg_dot import dot_2d
from aicssegmentation.core.utils import topology_preserving_thinning
from aicssegmentation.core.seg_dot import dot_3d_wrapper
from tifffile import imsave

import pandas as pd

In [107]:
filename = "//allen/aics/microscopy/PRODUCTION/PIPELINE_8_1/5500000704_EE_2-01/aligned_split/5500000704_EE_2-01_AcquisitionBlock7_pt7_Scene-19_aligned.ome.tiff"
reader = AICSImage(filename) 
IMG = reader.data.astype(np.float32)
#IMG = np.transpose(IMG, (0,2,1,3,4))
#IMG = np.concatenate([IMG, IMG], 2)

print(IMG.shape)

Attempted file (//allen/aics/microscopy/PRODUCTION/PIPELINE_8_1/5500000704_EE_2-01/aligned_split/5500000704_EE_2-01_AcquisitionBlock7_pt7_Scene-19_aligned.ome.tiff) load with reader: aicsimageio.readers.bfio_reader.OmeTiledTiffReader failed with error: No module named 'bfio'
The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).


(1, 4, 40, 1200, 1800)


In [108]:
N_CHANNELS = IMG.shape[2]
print(N_CHANNELS)
# MID_SLICE = np.int(0.5*IMG.shape[2])

# fig, ax = plt.subplots(1, N_CHANNELS, figsize=(18,16), dpi=72, facecolor='w', edgecolor='k')
# if N_CHANNELS==1:
#     ax.axis('off')
#     ax.imshow(IMG[0,0,0,MID_SLICE,:,:], cmap=plt.cm.gray)
# else:
#     for channel in range(N_CHANNELS):
#         ax[channel].axis('off')
#         ax[channel].imshow(IMG[0,0,channel,MID_SLICE,:,:], cmap=plt.cm.gray)

40


In [109]:
structure_channel = 1
#####################

struct_img0 = IMG[0,structure_channel,:,:,:].copy()
#view(single_fluorescent_view(struct_img0))

In [110]:
################################
## PARAMETERS for this step ##
intensity_scaling_param = [0]
################################
# intensity normalization/
struct_img = intensity_normalization(struct_img0, scaling_param=intensity_scaling_param)

# smoothing with edge preserving smoothing 
structure_img_smooth = edge_preserving_smoothing_3d(struct_img)

intensity normalization: min-max normalization with NO absoluteintensity upper bound


In [111]:
################################
## PARAMETERS for this step ##
f3_param = [[0.8, 0.02]] # good options: 0.03, 0.02
################################

bw = filament_3d_wrapper(structure_img_smooth, f3_param)

In [112]:
################################
## PARAMETERS for this step ##
minArea = 4
################################

seg = remove_small_objects(bw>0, min_size=minArea, connectivity=1, in_place=False)

In [113]:
# save the file
out=seg.astype(np.uint8)
out[out>0]=255
imsave("tuba1b-50ms-b7-sc19.tiff", out)